<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

The following is an example with the Ishigami function [6]

$$ Y(\mathbf{x}) = \sin x_1 + 7 \sin^2 x_2 + 0.1 x_3^4 \sin x_1, $$
with $\mathbf{x} \in [-\pi, \pi]^3$. This function exhibits strong
non-linearity and non-monotonicity.

Remember, Sobol' indices assumes that samples are independently
distributed. In this case we use a uniform distribution on each marginals.


In [ ]:
import numpy as np
from scipy.stats import sobol_indices, uniform
rng = np.random.default_rng()
def f_ishigami(x):
    f_eval = (
        np.sin(x[0])
        + 7 * np.sin(x[1])**2
        + 0.1 * (x[2]**4) * np.sin(x[0])
    )
    return f_eval
indices = sobol_indices(
    func=f_ishigami, n=1024,
    dists=[
        uniform(loc=-np.pi, scale=2*np.pi),
        uniform(loc=-np.pi, scale=2*np.pi),
        uniform(loc=-np.pi, scale=2*np.pi)
    ],
    random_state=rng
)
indices.first_order

array([0.31637954, 0.43781162, 0.00318825])

In [ ]:
indices.total_order

array([0.56122127, 0.44287857, 0.24229595])

Confidence interval can be obtained using bootstrapping.


In [ ]:
boot = indices.bootstrap()

Then, this information can be easily visualized.


In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(1, 2, figsize=(9, 4))
_ = axs[0].errorbar(
    [1, 2, 3], indices.first_order, fmt='o',
    yerr=[
        indices.first_order - boot.first_order.confidence_interval.low,
        boot.first_order.confidence_interval.high - indices.first_order
    ],
)
axs[0].set_ylabel("First order Sobol' indices")
axs[0].set_xlabel('Input parameters')
axs[0].set_xticks([1, 2, 3])
_ = axs[1].errorbar(
    [1, 2, 3], indices.total_order, fmt='o',
    yerr=[
        indices.total_order - boot.total_order.confidence_interval.low,
        boot.total_order.confidence_interval.high - indices.total_order
    ],
)
axs[1].set_ylabel("Total order Sobol' indices")
axs[1].set_xlabel('Input parameters')
axs[1].set_xticks([1, 2, 3])
plt.tight_layout()
plt.show()

.. note
```

By default, `scipy.stats.uniform` has support ``[0, 1]``.
Using the parameters ``loc`` and ``scale``, one obtains the uniform
distribution on ``[loc, loc + scale]``.

```
This result is particularly interesting because the first order index
$S_{x_3} = 0$ whereas its total order is $S_{T_{x_3}} = 0.244$.
This means that higher order interactions with $x_3$ are responsible
for the difference. Almost 25% of the observed variance
on the QoI is due to the correlations between $x_3$ and $x_1$,
although $x_3$ by itself has no impact on the QoI.

The following gives a visual explanation of Sobol' indices on this
function. Let's generate 1024 samples in $[-\pi, \pi]^3$ and
calculate the value of the output.


In [ ]:
from scipy.stats import qmc
n_dim = 3
p_labels = ['$x_1$', '$x_2$', '$x_3$']
sample = qmc.Sobol(d=n_dim, seed=rng).random(1024)
sample = qmc.scale(
    sample=sample,
    l_bounds=[-np.pi, -np.pi, -np.pi],
    u_bounds=[np.pi, np.pi, np.pi]
)
output = f_ishigami(sample.T)

Now we can do scatter plots of the output with respect to each parameter.
This gives a visual way to understand how each parameter impacts the
output of the function.


In [ ]:
fig, ax = plt.subplots(1, n_dim, figsize=(12, 4))
for i in range(n_dim):
    xi = sample[:, i]
    ax[i].scatter(xi, output, marker='+')
    ax[i].set_xlabel(p_labels[i])
ax[0].set_ylabel('Y')
plt.tight_layout()
plt.show()

Now Sobol' goes a step further:
by conditioning the output value by given values of the parameter
(black lines), the conditional output mean is computed. It corresponds to
the term $\mathbb{E}(Y|x_i)$. Taking the variance of this term gives
the numerator of the Sobol' indices.


In [ ]:
mini = np.min(output)
maxi = np.max(output)
n_bins = 10
bins = np.linspace(-np.pi, np.pi, num=n_bins, endpoint=False)
dx = bins[1] - bins[0]
fig, ax = plt.subplots(1, n_dim, figsize=(12, 4))
for i in range(n_dim):
    xi = sample[:, i]
    ax[i].scatter(xi, output, marker='+')
    ax[i].set_xlabel(p_labels[i])
    for bin_ in bins:
        idx = np.where((bin_ <= xi) & (xi <= bin_ + dx))
        xi_ = xi[idx]
        y_ = output[idx]
        ave_y_ = np.mean(y_)
        ax[i].plot([bin_ + dx/2] * 2, [mini, maxi], c='k')
        ax[i].scatter(bin_ + dx/2, ave_y_, c='r')
ax[0].set_ylabel('Y')
plt.tight_layout()
plt.show()

Looking at $x_3$, the variance
of the mean is zero leading to $S_{x_3} = 0$. But we can further
observe that the variance of the output is not constant along the parameter
values of $x_3$. This heteroscedasticity is explained by higher order
interactions. Moreover, an heteroscedasticity is also noticeable on
$x_1$ leading to an interaction between $x_3$ and $x_1$.
On $x_2$, the variance seems to be constant and thus null interaction
with this parameter can be supposed.

This case is fairly simple to analyse visually---although it is only a
qualitative analysis. Nevertheless, when the number of input parameters
increases such analysis becomes unrealistic as it would be difficult to
conclude on high-order terms. Hence the benefit of using Sobol' indices.
